# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers every time.

In [3]:
llm_model = os.getenv('LLM_MODEL')

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [6]:
## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product = "Queen Size Sheet Set"
chain.run(product)

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"The best name for a company that specializes in making queen size sheet sets could encompass the idea of comfort, luxury, sleep, or the specific product focus on queen size bedding. Here are some suggestions:\n\n1. Queen's Comfort Linens\n2. Royal Slumber Sheets\n3. Deluxe Dreams Bedding\n4. Queen's Embrace Textiles\n5. Regal Rest Collections\n6. Elegant Night's Queen Sheets\n7. Serenity Sleepwear & Linens\n8. Majestic Bed Threads\n9. Sovereign Sheets Co.\n10. Queenly Quilts & Sheets\n\nIdeally, the name should be easy to remember, reflect the product's quality, and give a sense of the specialization in queen size sheet sets. Ensure that the chosen name is also unique within the industry to avoid trademark issues and to stand out in the market."

## SequentialChain

![](../images/09-chain.png)

In [12]:
from langchain.chains import SequentialChain

![](../images/10-simple-seq-chain.png)

In [13]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [16]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [17]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

![](../images/11-seq-chain.png)

In [18]:
review = df.Review[5]
overall_chain(review)

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or fake!?",
 'summary': 'The reviewer is disappointed with the mediocre taste and poor foam quality of the product, suggesting it might be an old batch or counterfeit compared to the superior versions bought in stores.',
 'followup_message': "Réponse de suivi:\n\nCher critique,\n\nNous sommes sincèrement désolés d'apprendre que votre expérience avec notre produit n'a pas été à la hauteur de vos attentes. La qualité de nos produits est une priorité absolue, et ce que vous décrivez ne reflète certainement pas nos normes habituelles. Nous prenons vos commentaires au sérieux et souhaitons enquêter davantage sur cette question.\n\nPourriez-vous nous fournir le num

## Router Chain

![](../images/12-router-chain.png)

In [19]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [20]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [21]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [22]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [25]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [26]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [27]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [28]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an idealized object called a black body. A black body is an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. That is, it is a perfect absorber of light and other forms of electromagnetic radiation.\n\nThe interesting thing about a black body is that it is also a perfect emitter of radiation. The radiation emitted by a black body is characterized by a spectrum that depends only on the body's temperature, a distribution known as the Planck radiation law. This spectrum has a specific peak wavelength that shifts to shorter wavelengths as the temperature increases, which is described by Wien's displacement law.\n\nThe concept of black body radiation is fundamental in the field of thermodynamics and quantum mechanics, as it led to the development of the quantum hypothesis by Max Planck. Planck's work on black body radiation was pivotal in the dev

In [29]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to your question is simple: 2 + 2 equals 4.'

In [30]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because it is the blueprint for building and maintaining the organism. DNA, or deoxyribonucleic acid, carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses.\n\nHere are several reasons why DNA is present in every cell:\n\n1. Genetic Information: DNA contains the instructions needed to build the proteins that make up our body and carry out various functions. Each gene within the DNA sequence encodes information for the production of a specific protein.\n\n2. Cell Division: When cells divide, they need to pass on their genetic information to their daughter cells. This ensures that each new cell has the same genetic material as the parent cell, allowing it to function properly.\n\n3. Development and Differentiation: As a multicellular organism develops from a single fertilized egg, cells divide and differentiate into various types with specialized functions. DNA gu